# Uporaba razdalj in podobnosti

V tem zvezku predstavimo, kako med prej dobljenimi vložitvami dokumentov računamo in uporabljamo razdalje in podobnosti.

Za začetek si spet preko API-ja pridobimo besedila vseh predlogov vladi, ki vsebujejo vsaj 50 znakov ter iz datotek preberemo fastText in tf-idf vektorje iz prejšnjega primera.

In [1]:
from textsemantics.server_api import ServerAPI
import numpy as np
from scipy.sparse import load_npz

api = ServerAPI()
datasets = api.list_datasets()
metadata = api.get_metadata(datasets[2][0])

texts = api.get_texts(urls=metadata['text'])
texts = [text for text in texts if len(text) > 50]

ft = np.load('data/ft.npy')
tfidf = load_npz('data/tfidf.npz').toarray()
word_embs = np.load('data/words.npy')

## Računanje razdalj med dokumenti

Razdaljo med dvema dokumentoma izračunamo kot razdaljo med njihovima vektorskima predstavitvama. Ker imamo mehanizem za računanje razdalj, lahko za želeni dokument poiščemo k dokumentov, ki so mu najbližji v vektorskem prostoru. Poiščimo 3 najbližje sosede prvega dokumenta v našem seznamu.

In [2]:
def cos_sim(x, y):
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

def knn(embs, ind=0, k=3):
    dists = [1 - cos_sim(embs[i], embs[ind]) if i != ind else 1
             for i in range(embs.shape[0])]
    inds = np.argsort(dists)
    return inds[:min(k, embs.shape[0])]

In [3]:
ft_nn = knn(ft, ind=0, k=3)
tfidf_nn = knn(tfidf, ind=0, k=3)

print(f'ID treh najbližjih sosedov za fastText {list(ft_nn)}')
print(f'ID treh najbližjih sosedov za fastText {list(tfidf_nn)}')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in float_scalars
  


ID treh najbližjih sosedov za fastText [2466, 7003, 8150]
ID treh najbližjih sosedov za fastText [2466, 8150, 7003]


Izkaže se, da z uporabo obeh vložitev odkrijemo iste 3 najbližje sosede. Poglejmo besedila prvega dokumenta in njegovih najbližjih sosedov.

In [4]:
from IPython.display import Markdown, display
display(Markdown(
    f"## Originalni dokument\n{texts[0]}\n\n ## 1. dokument\n{texts[ft_nn[0]]}\n\n" +
    f"## 2. dokument\n{texts[ft_nn[1]]}\n\n ## 3. dokument\n{texts[ft_nn[2]]}\n\n"))


## Originalni dokument
 

Predlagam, da se na slovenskih semaforjih znova uvede utripajoča zelena luč, kot smo jo že imeli pred leti.  Prednost utripajoče zelene luči je v tem, da voznike predhodno opozori, da se bo prižgala rumena in se morajo na semaforju ustaviti. Mislim, da bodo s tem slovenske ceste varnejše ter promet bolj tekoč.
 
Add1.
Predlagam tudi, da se sprejme načrt uvedbe odštevalnikov na semaforjih, po katerem bi do konca leta 2012 20% vseh semaforjev v državi imelo odštevalnike (tako za zeleno kot za rdečo luč).

 ## 1. dokument
Predlagam, da se odštevanje časa prižgane zelene luči na semaforju ukine.
 
Da so odštevalniki zelene luči  namreč podobno nevarni kot utripajoča zelena luč so ugotovili v Tajvanu (že leta 2008), kjer so na podlagi raziskave tudi uvedli spremembo zakona, ki ureja prometno signalizacijo.
http://freakonomics.blogs.nytimes.com/2008/03/14/taiwans-solution-to-t...
http://www.taipeitimes.com/News/taiwan/archives/2008/03/11/2003405001
http://www.traffictechnologytoday.com/news.php?NewsID=4104
 
Odštevalniki zelene luči so po učinku enaki utripajoči zeleni luči, o tej pa je bilo na predlagam.vladi.si že govora. Takrat je pristojni organ odgovoril:
 
In nadalje:
 
 
 
Ker se torej vozniška praksa oblikuje po več letih in ker imajo odštevalniki ob prižgani zeleni luči na semaforju, podobno kot utripajoča zelena luč, na varnost negativen učinek, je v interesu vseh, da se odštevalnike ob zeleni luči tudi zakonsko prepove, preden dosežejo širšo uporabo.
Odštevalniki ob rdeči luči pa imajo na voznike pomirjujoč učinek in pozitivno vplivajo na varnost v prometu, nesmiselno pa bi jih bilo zahtevati za vsak semafor, zato lahko ostanejo neobvezni oz. njihove rabe ni potrebno zakonsko omejevati.

## 2. dokument
Spoštovani,
Predlagam, da se na semaforiziranih cestah, kjer je dovoljena omejitev višja od 50 km/h, uvede utripajoča zelena luč. Pred časom sem bila v Budimpešti, kjer zelena luč 3x utripne preden se prižge oranžna. Utripajoča zelena luč je pravi "balzam" za voznika, saj je dovolj časa, da se odloči, ali bo upočasnil ali pa pospešil in speljal preko zelene / oranžne luči. Pri nas pa smo z ukinitvijo utripajoče zelene luči povzročili napetost pri voznikih saj imamo 2 opciji: ali divje zaviramo, ali pa prevozimo oranžno /  rdečo luč. Zavedati se moramo, da na cestah, kjer je hitrost višja od 50 km/h ima voznik manj časa za razmišljanje in če bi uvedli utripajočo zeleno luč bi pridobili dovolj časa za pravilno odločitev in manj stresa.

 ## 3. dokument
Predlagam vladi RS, da namesti ponovno odštevanje časa, ko se bo prižgala po zeleni oranžna luč na semaforju.  Nekateri semaforji v Ljubljani to imajo.  Ker točno vidiš, kdaj se bo po zeleni prižgala oranžna luč in je za voznika boljše, ker pričakuje spremembo in se pripravi.



Vsi štirje predlogi se nanašajo na odštevalnike in utripajoče luči pri semaforjih. Na takšen enostaven način lahko za poljuben predlog vladi poiščemo obstoječe predloge, ki so mu podobni.

## Podobnost med dokumentom in besedo

Predpostavimo, da imamo seznam besed, za katere želimo vedeti, ali dobro opisujejo določeni dokument. To lahko izvemo na način podoben zgoraj opisanemu. Ker smo z modelom fastText dobili vložitve besed, nato pa le te združili v vložitve dokumentov, so tako besede kot dokumenti vektorji v istem vektorskem prostoru. Zato lahko računamo razdalje (podobnosti) med besedami in dokumenti. Vemo, da se prvi predlog vladi na seznamu nanaša na semaforje. Izračunajmo kosinusno podobnost (definirana je kot 1 - razdalja) med tem predlogom in besedami semafor, luč, zdravje in tehnologija.

In [5]:
for i, word in enumerate(['semafor', 'luč', 'zdravje', 'tehnologija']):
    print(f"Podobnost med dokumentom in besedo {word}: " + 
          f"{round(cos_sim(ft[0], word_embs[i, :]), 4)}")

Podobnost med dokumentom in besedo semafor: 0.6116999983787537
Podobnost med dokumentom in besedo luč: 0.5444999933242798
Podobnost med dokumentom in besedo zdravje: 0.3411000072956085
Podobnost med dokumentom in besedo tehnologija: 0.3946000039577484


Kot vidimo, sta besedi semafor in luč bolj podobni dokumentu kot besedi zdravje in tehnologija.